In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_horizontal_wavelet_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,170560.76,-1.02,-0.10,0.55,0.00,-0.06,0.01,-0.0,-0.00
3,-1.02,35109.95,-0.20,0.09,-0.08,0.01,0.02,0.0,0.00
4,-0.10,-0.20,7050.08,0.10,-0.00,0.00,-0.01,-0.0,0.00
5,0.55,0.09,0.10,1374.67,0.00,-0.01,0.01,0.0,0.00
6,0.00,-0.08,-0.00,0.00,254.82,-0.00,-0.00,0.0,0.00
7,-0.06,0.01,0.00,-0.01,-0.00,45.32,0.00,0.0,0.00
8,0.01,0.02,-0.01,0.01,-0.00,0.00,7.60,0.0,0.00
9,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,1.2,-0.00
10,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.0,0.15


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,-0.00000,0.00004,0.00000,-0.00002,0.00001,-0.00001,-0.00000
3,-0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00001,0.00004,0.00000,0.00001
4,-0.00000,-0.00001,1.00000,0.00003,-0.00000,0.00000,-0.00003,-0.00001,0.00001
5,0.00004,0.00001,0.00003,1.00000,0.00000,-0.00002,0.00005,0.00000,0.00001
6,0.00000,-0.00003,-0.00000,0.00000,1.00000,-0.00001,-0.00000,0.00002,0.00000
7,-0.00002,0.00001,0.00000,-0.00002,-0.00001,1.00000,0.00002,0.00000,0.00000
8,0.00001,0.00004,-0.00003,0.00005,-0.00000,0.00002,1.00000,0.00003,0.00000
9,-0.00001,0.00000,-0.00001,0.00000,0.00002,0.00000,0.00003,1.00000,-0.00001
10,-0.00000,0.00001,0.00001,0.00001,0.00000,0.00000,0.00000,-0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1.6796495335261714

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.23465659e+09 1.34855332e+08 5.32670442e+06 1.96913837e+05
 6.49772066e+03 1.92808081e+02 4.83364680e+00 9.10516351e-02
 3.26055434e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,9.996260e-01,-0.026841,-0.005142,-0.000998,-0.000185,-0.000033,-0.000005,-8.940122e-07,-1.086083e-07
1,2.666900e-02,0.999178,-0.030004,-0.005587,-0.001037,-0.000184,-0.000030,-4.865959e-06,-5.847915e-07
2,5.899464e-03,0.029644,0.998950,-0.033891,-0.006017,-0.001068,-0.000180,-2.842643e-05,-3.423621e-06
3,1.336058e-03,0.006511,0.033441,0.998653,-0.038534,-0.006584,-0.001099,-1.742137e-04,-2.119584e-05
4,2.973568e-04,0.001447,0.007204,0.037952,0.998162,-0.046066,-0.007431,-1.171836e-03,-1.430125e-04
5,6.633899e-05,0.000320,0.001597,0.008200,0.045260,0.997236,-0.057651,-8.780246e-03,-1.069784e-03
6,1.464881e-05,0.000071,0.000356,0.001819,0.009840,0.056339,0.995034,-8.088436e-02,-9.503849e-03
7,3.473169e-06,0.000017,0.000084,0.000428,0.002312,0.013008,0.078491,9.893050e-01,-1.222291e-01
8,8.563770e-07,0.000004,0.000021,0.000106,0.000571,0.003210,0.019132,1.210568e-01,9.924558e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00037402851564782846,
 0.0008222509359203656,
 0.0010503411160309728,
 0.0013468844415628345,
 0.0018382651366508629,
 0.0027639450217379347,
 0.004965829352111872,
 0.01069495532942466,
 0.007544171878620443]